<a href="https://colab.research.google.com/github/anna-boser/AutoGluon_PM_paper/blob/main/code/ML_model_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up Autogluon and get data

In [ ]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools
!pip install -U "mxnet<2.0.0"
# Install pre-release, frozen to a particual pre-release for stability
!pip install --pre "autogluon==0.0.16b20201214"
!pip install -U ipykernel

In [ ]:
import autogluon as ag
from autogluon.tabular import TabularPrediction as task
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import gc
import math
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NASA PM_estimation/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("Data/datasets/Train.csv")
df.head()

,Unnamed: 0,Unique,Id,Day,Lat,Lon,Elevation,Emissions,Forest,Roads,Streets,Plumes_High,Plumes_Med,Plumes_Low,Max_Temp,Max_Wind,Precip,Rel_Humidity,Wind_Dir,BLH,AOD,PM
0,1,1131_1,1131,1,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,53.4,5.3,0.0,67.0,251.0,758.111816,NaN,3.7
1,2,1131_100,1131,100,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,64.1,3.6,0.0,48.0,308.0,1155.824219,NaN,2.6
2,3,1131_101,1131,101,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,62.7,5.1,0.0,64.0,200.0,1162.292725,NaN,2.9
3,4,1131_102,1131,102,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,67.7,7.9,0.0,68.0,221.0,263.273407,NaN,0.5
4,5,1131_104,1131,104,36.840574,-121.366314,122.986002,0.0,0,0.0,0.191756,0,0,0,60.1,6.5,0.0,44.0,314.0,1268.919434,0.097667,2.4


# Crossvalidate 
Make tables of crossvalidated predictions by station for various models \\
\
Models chosen are those from Autogluon default: \\
\
hyperparameters = {
‘NN’: {}, ‘GBM’: [

{}, {‘extra_trees’: True, ‘AG_args’: {‘name_suffix’: ‘XT’}},

], ‘CAT’: {}, ‘RF’: [

{‘criterion’: ‘gini’, ‘AG_args’: {‘name_suffix’: ‘Gini’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘entropy’, ‘AG_args’: {‘name_suffix’: ‘Entr’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘mse’, ‘AG_args’: {‘name_suffix’: ‘MSE’, ‘problem_types’: [‘regression’]}},

], ‘XT’: [

{‘criterion’: ‘gini’, ‘AG_args’: {‘name_suffix’: ‘Gini’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘entropy’, ‘AG_args’: {‘name_suffix’: ‘Entr’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘mse’, ‘AG_args’: {‘name_suffix’: ‘MSE’, ‘problem_types’: [‘regression’]}},

], ‘KNN’: [

{‘weights’: ‘uniform’, ‘AG_args’: {‘name_suffix’: ‘Unif’}}, {‘weights’: ‘distance’, ‘AG_args’: {‘name_suffix’: ‘Dist’}},

], ‘custom’: [‘GBM’]

}

In [ ]:
def cv(hyperparameters):
  n_fold = len(set(df['Id']))
  kf = GroupKFold(n_fold)
  split = kf.split(df, groups = df['Id'])

  metric_df = pd.DataFrame(columns = ['station', 'Day', 'PM', 'PM_pred', 'rmse', 'bias'])

  for i, (train_idx, test_idx) in enumerate(split):
    print(f'Starting training fold {i}.')
    _ = gc.collect()

    features = ['Day', 'Lat', 'Lon', 'Elevation', 'Emissions', 'Forest',
        'Roads', 'Streets', 'Plumes_High', 'Plumes_Med', 'Plumes_Low',
        'Max_Temp', 'Max_Wind', 'Precip', 'Rel_Humidity', 'Wind_Dir', 'BLH',
        'AOD']
    label_column = 'PM'

    X = df[features]
    y = df[label_column]

    train_data = task.Dataset(pd.concat([X.loc[train_idx], y.loc[train_idx]], axis=1))
    test_data = task.Dataset(pd.concat([X.loc[test_idx], y.loc[test_idx]], axis=1))

    predictor = task.fit(train_data=train_data, 
                         label=label_column, 
                         hyperparameters=hyperparameters, 
                         # time_limits = 60*60, # un-comment these for the tuned neural network. Train for 1 hour. 
                         # hyperparameter_tune=True, 
                         eval_metric='r2')

    y_test = test_data[label_column]

    test_data_nolab = test_data.drop(labels = [label_column], axis = 1)

    y_pred = predictor.predict(test_data_nolab)
    
    bias = y_pred-y_test
    rmse = np.abs(bias)

    station = np.repeat(df.loc[test_idx]['Id'].iloc[0], len(test_data['Day']))
    df_to_append = pd.DataFrame({'station': station, 'Day': test_data['Day'], 'PM':y_test, 'PM_pred': y_pred, 'rmse': rmse, 'bias':bias})

    metric_df = metric_df.append(df_to_append, ignore_index = True)

  return metric_df

In [ ]:
CAT = cv({'CAT': {}})
CAT.to_csv("Data/output/CV/CAT.csv", index = False)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210506_223001/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210506_223001/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.76949, 7.76697)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])


Starting training fold 0.


Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12098.71 MB
	Train Data (Original)  Memory Usage: 1.38 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 

Starting training fold 1.


Fitting model: CatBoost ...
	0.8423	 = Validation r2 score
	15.43s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8423	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 15.77s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210506_223034/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210506_223034/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.85419, 7.72596)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Fea

...

		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.41 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.16s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify

Starting training fold 32.


Fitting model: CatBoost ...
	0.8309	 = Validation r2 score
	11.29s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8309	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 11.71s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210506_223709/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210506_223709/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9846
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.82056, 7.71767)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Fea

Starting training fold 33.


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.42 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.17s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To

In [ ]:
NN = cv({'NN': {}})
NN.to_csv("Data/output/CV/NN.csv", index = False)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210506_224712/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210506_224712/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.76949, 7.76697)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11772.15 MB
	Train Data (Original)  Memory Usage: 1.38 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

Starting training fold 0.


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.38 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.13s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric argument of fit()
A

Starting training fold 1.


Fitting model: NeuralNetMXNet ...
	0.6402	 = Validation r2 score
	71.18s	 = Training runtime
	0.15s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.6402	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 71.81s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210506_224921/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210506_224921/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.85419, 7.72596)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Usi

...

	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.41 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.17s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: 'r2'
Fitting model: NeuralNetMXNet ...
	0.6904	 = Validation r2 score
	70.92s	 = Training runtime
	0.15s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.6904	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 71.58s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210506_232116/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210506_232116/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9802
Train Data Columns: 18
Preprocessing

Starting training fold 32.


	Available Memory:                    11759.96 MB
	Train Data (Original)  Memory Usage: 1.41 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data u

Starting training fold 33.


	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.42 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.17s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: 'r2'
Fitting model: NeuralNetMXNet ...
	0.6567	 = Validation r2 score
	98.81s	 = Training runtime
	0.14s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.6567	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 99.5s ...


In [ ]:
GBM = cv({'GBM': {}})
GBM.to_csv("Data/output/CV/GBM.csv", index = False)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_000702/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_000702/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.76949, 7.76697)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11762.77 MB
	Train Data (Original)  Memory Usage: 1.38 MB (0.0% of available memory)


Starting training fold 0.


	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.38 MB (0.0% of available memory)
Dat

Starting training fold 1.


Fitting model: LightGBM ...
	0.8122	 = Validation r2 score
	1.14s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8122	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1.5s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_000708/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_000708/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.85419, 7.72596)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Featur

...

	Available Memory:                    11754.24 MB
	Train Data (Original)  Memory Usage: 1.41 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data u

Starting training fold 32.


		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.41 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.17s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: 'r2'
Fitting model: LightGBM ...
	0.8185	 = Validation r2 score
	3.12s	 = Training runtime
	0.09s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8185	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training 

Starting training fold 33.


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.42 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.16s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To

In [ ]:
RF = cv({'RF': {}})
RF.to_csv("Data/output/CV/RF.csv", index = False)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_000920/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_000920/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.76949, 7.76697)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11759.8 MB
	Train Data (Original)  Memory Usage: 1.38 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to m

Starting training fold 0.


		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.38 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.14s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric argument of fit()
AutoGluon will early s

Starting training fold 1.


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.38 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.13s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric argument of fit()
A

...

Fitting model: RandomForest ...
	0.6988	 = Validation r2 score
	17.08s	 = Training runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.6988	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 19.52s ...


Starting training fold 32.


No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_002002/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_002002/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9802
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.836, 7.73231)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11717.69 MB
	Train Data (Original)  Memory Usage: 1.41 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to ma

Starting training fold 33.


Fitting model: RandomForest ...
	0.7427	 = Validation r2 score
	17.27s	 = Training runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.7427	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 20.05s ...


In [ ]:
XT = cv({'XT': {}})
XT.to_csv("Data/output/CV/XT.csv", index = False)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_002516/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_002516/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.76949, 7.76697)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11703.07 MB
	Train Data (Original)  Memory Usage: 1.38 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

Starting training fold 0.


Fitting model: ExtraTrees ...
	0.7693	 = Validation r2 score
	7.35s	 = Training runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.7693	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 11.62s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_002529/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_002529/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.78561, 7.64778)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Fea

Starting training fold 1.


Fitting model: ExtraTrees ...
	0.7831	 = Validation r2 score
	8.4s	 = Training runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.7831	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 11.78s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_002541/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_002541/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.85419, 7.72596)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feat

...

	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.41 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.15s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To

Starting training fold 32.


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.41 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.14s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To

Starting training fold 33.


	Train Data (Original)  Memory Usage: 1.42 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Tr

In [ ]:
KNN = cv({'KNN': {}})
KNN.to_csv("Data/output/CV/KNN.csv", index = False)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_003255/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_003255/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.76949, 7.76697)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11687.89 MB
	Train Data (Original)  Memory Usage: 1.38 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

Starting training fold 0.


Fitting model: KNeighbors ...
	0.1739	 = Validation r2 score
	0.06s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.1739	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.6s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_003256/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_003256/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.78561, 7.64778)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Featu

Starting training fold 1.


Fitting model: KNeighbors ...
	0.2182	 = Validation r2 score
	0.04s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.2182	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.55s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_003257/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_003257/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.85419, 7.72596)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feat

...

Fitting model: KNeighbors ...
	0.195	 = Validation r2 score
	0.05s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.195	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.62s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_003324/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_003324/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9802
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.836, 7.73231)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature 

Starting training fold 32.


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.41 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.15s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To

Starting training fold 33.


Fitting model: KNeighbors ...
	0.1237	 = Validation r2 score
	0.05s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.1237	 = Validation r2 score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.89s ...


In [ ]:
AutoGluon = cv('default')
AutoGluon.to_csv("Data/output/CV/AutoGluon.csv", index = False)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_003345/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_003345/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.76949, 7.76697)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11680.77 MB
	Train Data (Original)  Memory Usage: 1.38 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to 

Starting training fold 0.


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.38 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.14s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric argument of fit()
A

[1000]	train_set's l2: 0.121002	train_set's r2: 0.99675	valid_set's l2: 7.54345	valid_set's r2: 0.829629


	0.8335	 = Validation r2 score
	10.34s	 = Training runtime
	0.15s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8563	 = Validation r2 score
	0.51s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 144.31s ...
No output_directory specified. Models will be saved in: AutogluonModels/ag-20210507_003610/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210507_003610/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    9573
Train Data Columns: 18
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.78561, 7.64778)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preproc

...

	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.42 MB (0.0% of available memory)
Dat

[1000]	train_set's l2: 3.18139	train_set's r2: 0.945785	valid_set's l2: 10.8647	valid_set's r2: 0.798245


	0.8011	 = Validation r2 score
	6.65s	 = Training runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost ...
	0.8554	 = Validation r2 score
	11.21s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ...
	0.8277	 = Validation r2 score
	14.35s	 = Training runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	0.68	 = Validation r2 score
	75.53s	 = Training runtime
	0.17s	 = Validation runtime
Fitting model: LightGBMCustom ...


[1000]	train_set's l2: 0.127107	train_set's r2: 0.996502	valid_set's l2: 10.3851	valid_set's r2: 0.805514


	0.8086	 = Validation r2 score
	11.02s	 = Training runtime
	0.19s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	0.8558	 = Validation r2 score
	0.46s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 158.3s ...


# Consolidate dataframes

In [ ]:
small_ids = pd.read_csv("Data/datasets/Small_Ids.csv")['x']
small_ids = [float(i) for i in small_ids]

full_df = pd.DataFrame(columns = ['Model', 'test_area', 'station', 'Day', 'PM', 'PM_pred'])

paths = glob.glob("Data/output/CV/*.csv")

for path in paths:
  f = os.path.split(path)[1]
  Model = os.path.splitext(f)[0]
  Table = pd.read_csv(path)
  Table['test_area'] = Table['station'].isin(small_ids)
  Table['Model'] = Model

  full_df = full_df.append(Table, ignore_index = True)

full_df.to_csv("Data/output/all_cv_preds.csv", index = False)